# FourSquare API-San Francisco City Neighborhood Analysis with Python and Folium

In [1]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
from config_foursquare import CLIENT_ID,CLIENT_SECRET
CLIENT_ID =CLIENT_ID
CLIENT_SECRET = CLIENT_SECRET 
VERSION="20180731"
LIMIT=30


import warnings #Warning module to filter any future warnings
warnings.filterwarnings('ignore')

In [3]:
address = '501 Geary St, San Francisco, CA 94102'#The Marker SFO
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.7866329 -122.41195210013


In [4]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [5]:
url=f"http://api.foursquare.com/v2/venues/search?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={latitude},{longitude}&query={search_query}&radius={radius}&limit=10"
url

'http://api.foursquare.com/v2/venues/search?client_id=L0XUK3OF0DEMWCIMYXLNQEDWMM1GIBC43PQCFYYID2XGX4WV&client_secret=FYVOYA1ZZ3YQ320RWOU5XCVD1Y3TA1BCTGIW2OS5ZJTG4CKH&v=20180731&ll=37.7866329,-122.41195210013&query=Italian&radius=500&limit=10'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b6385d74c1f677b0b6d06c6'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'hasPerk': False,
    'id': '4a19721ef964a520197a1fe3',
    'location': {'address': '624 Post St',
     'cc': 'US',
     'city': 'San Francisco',
     'country': 'United States',
     'crossStreet': 'btw Taylor St & Trader Vic Aly',
     'distance': 146,
     'formattedAddress': ['624 Post St (btw Taylor St & Trader Vic Aly)',
      'San Francisco, CA 94109',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.78792121397233,
       'lng': -122.41226661901916}],
     'lat': 37.78792121397233,
     'lng': -122.41226661901916,
     'postalCode': '94109',
     'state': 'CA'},


In [7]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4a19721ef964a520197a1fe3,624 Post St,US,San Francisco,United States,btw Taylor St & Trader Vic Aly,146,"[624 Post St (btw Taylor St & Trader Vic Aly),...","[{'label': 'display', 'lat': 37.78792121397233...",37.787921,-122.412267,94109,CA,Fino Restaurant,v-1533248983
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4af6357bf964a520400222e3,835 Hyde St,US,San Francisco,United States,NaN,495,"[835 Hyde St, San Francisco, CA 94109, United ...","[{'label': 'display', 'lat': 37.78871293006333...",37.788713,-122.416926,94109,CA,Colombini Bistro,v-1533248983
2,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",False,4f44ee5819836ed001978c46,870 Market St,US,San Francisco,United States,NaN,486,"[870 Market St, San Francisco, CA 94102, Unite...","[{'label': 'display', 'lat': 37.785415, 'lng':...",37.785415,-122.406645,94102,CA,Panoroma Italiano,v-1533248983


In [8]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
print(filtered_columns)
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

['name', 'categories', 'location.address', 'location.cc', 'location.city', 'location.country', 'location.crossStreet', 'location.distance', 'location.formattedAddress', 'location.labeledLatLngs', 'location.lat', 'location.lng', 'location.postalCode', 'location.state', 'id']


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fino Restaurant,Italian Restaurant,624 Post St,US,San Francisco,United States,btw Taylor St & Trader Vic Aly,146,"[624 Post St (btw Taylor St & Trader Vic Aly),...","[{'label': 'display', 'lat': 37.78792121397233...",37.787921,-122.412267,94109,CA,4a19721ef964a520197a1fe3
1,Colombini Bistro,Italian Restaurant,835 Hyde St,US,San Francisco,United States,NaN,495,"[835 Hyde St, San Francisco, CA 94109, United ...","[{'label': 'display', 'lat': 37.78871293006333...",37.788713,-122.416926,94109,CA,4af6357bf964a520400222e3
2,Panoroma Italiano,Food,870 Market St,US,San Francisco,United States,NaN,486,"[870 Market St, San Francisco, CA 94102, Unite...","[{'label': 'display', 'lat': 37.785415, 'lng':...",37.785415,-122.406645,94102,CA,4f44ee5819836ed001978c46


In [9]:
dataframe_filtered.name

0      Fino Restaurant
1     Colombini Bistro
2    Panoroma Italiano
Name: name, dtype: object

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='The Marker',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng ,label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [61]:
venue_id='4a19721ef964a520197a1fe3'
url_ven=f"https://api.foursquare.com/v2/venues/{venue_id}?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}"

In [64]:
result_ven=requests.get(url_ven).json()
result_ven

{'meta': {'code': 504,
  'errorDetail': 'Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.',
  'errorType': 'server_error'},
 'response': {}}

In [63]:
result_ven['response']['venue'].keys()

KeyError: 'venue'

In [40]:
try:
    print(result_ven['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [54]:
venueid='4af6357bf964a520400222e3'
url_ven=f"https://api.foursquare.com/v2/venues/{venueid}?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}"
result_ven=requests.get(url_ven).json()
try:
    print(result_ven['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [59]:
#tips_count=result_ven['response']['venue']['tips']['count']
result_ven['response']

{}

In [43]:
venueid='4f44ee5819836ed001978c46'
url_ven=f"https://api.foursquare.com/v2/venues/{venueid}?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}"
result_ven=requests.get(url_ven).json()
try:
    print(result_ven['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [47]:
result_ven['response']['venue']['tips']['count']

KeyError: 'venue'

## Get the venue's tips
https://api.foursquare.com/v2/venues/VENUE_ID/tips?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION&limit=LIMIT

In [20]:
VENUE_ID='4a19721ef964a520197a1fe3'
url_tips=f"https://api.foursquare.com/v2/venues/{VENUE_ID}/tips?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&limit={LIMIT}"

In [21]:
result_tips=requests.get(url_tips).json()

In [22]:
result_tips['response']['tips']['items'][0]['text']

'Delicious and authentic! I loved their risotto special with the HUGE scallops. Our table ordered many dishes and desserts and they all stood out.'

In [23]:
tips_df=json_normalize(result_tips['response']['tips']['items'][0])

In [24]:
tips_df

,agreeCount,authorInteractionType,canonicalUrl,createdAt,disagreeCount,id,lang,lastUpvoteTimestamp,lastVoteText,likes.count,...,photourl,text,todo.count,type,user.firstName,user.gender,user.id,user.lastName,user.photo.prefix,user.photo.suffix
0,1,liked,https://foursquare.com/item/5b0f202fcf72a0002c...,1527717935,0,5b0f202fcf72a0002cf16c2d,en,1530744170,Upvoted 4 weeks ago,0,...,https://igx.4sqi.net/img/general/original/7835...,Delicious and authentic! I loved their risotto...,0,user,Paige,female,7835045,C,https://igx.4sqi.net/img/user/,/7835045_vzeT9M7E_W9mOqaLB2Kjg0RdSk9ZaZ3qumWd8...


In [25]:
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Delicious and authentic! I loved their risotto...,1,0,5b0f202fcf72a0002cf16c2d,Paige,C,female,7835045


In [42]:
result_tips['response']['tips']['items'][0]

{'agreeCount': 1,
 'authorInteractionType': 'liked',
 'canonicalUrl': 'https://foursquare.com/item/5b0f202fcf72a0002cf16c2d',
 'createdAt': 1527717935,
 'disagreeCount': 0,
 'id': '5b0f202fcf72a0002cf16c2d',
 'lang': 'en',
 'lastUpvoteTimestamp': 1530744170,
 'lastVoteText': 'Upvoted 4 weeks ago',
 'likes': {'count': 0, 'groups': []},
 'logView': True,
 'photo': {'createdAt': 1527717937,
  'height': 1920,
  'id': '5b0f2031b9b37b002cda25de',
  'prefix': 'https://igx.4sqi.net/img/general/',
  'source': {'name': 'Foursquare for iOS',
   'url': 'https://foursquare.com/download/#/iphone'},
  'suffix': '/7835045_ZBAbXkjiSO-GfaOsZLWOHix3QJK9WaCBomOpKJz0aaM.jpg',
  'visibility': 'public',
  'width': 1440},
 'photourl': 'https://igx.4sqi.net/img/general/original/7835045_ZBAbXkjiSO-GfaOsZLWOHix3QJK9WaCBomOpKJz0aaM.jpg',
 'text': 'Delicious and authentic! I loved their risotto special with the HUGE scallops. Our table ordered many dishes and desserts and they all stood out.',
 'todo': {'count': 0

# Search a Foursquare User
https://api.foursquare.com/v2/users/USER_ID?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION

In [26]:
USER_ID="7835045"
url_user=f"https://api.foursquare.com/v2/users/{USER_ID}?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}"

In [27]:
result_user=requests.get(url_user).json()
user_data=result_user['response']['user']
result_user['response']['user'].keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'superuser', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [40]:
user_data=result_user['response']['user']
user_data

{'bio': 'I thrive on trying new things, visiting exotic places & noshing on delicious foods. Constantly looking for dog friendly spots that my pups will like.',
 'canonicalUrl': 'https://foursquare.com/paigeknowsfirst',
 'checkins': {'count': 18304, 'items': []},
 'contact': {},
 'firstName': 'Paige',
 'friends': {'count': 31,
  'groups': [{'count': 31,
    'items': [{'bio': '',
      'contact': {},
      'firstName': 'Jen',
      'gender': 'female',
      'homeCity': 'New York, NY',
      'id': '48519984',
      'lastName': 'Pace',
      'lists': {'groups': [{'count': 2, 'items': [], 'type': 'created'}]},
      'photo': {'prefix': 'https://igx.4sqi.net/img/user/',
       'suffix': '/48519984-I3FRIHLO1CXJVKB1.jpg'},
      'tips': {'count': 6}},
     {'bio': 'DJ. MC. Self-Proclaimed #GAMECHANGER. Polish #3BlocksFromCompton',
      'contact': {},
      'firstName': 'Eva',
      'gender': 'female',
      'homeCity': 'Los Angeles, CA',
      'id': '9876567',
      'lastName': 'Magazine',
 

In [28]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])
print('Count of Tips: '+str(user_data['tips']['count']))

First Name: Paige
Last Name: C
Home City: New York, NY
Count of Tips: 1863


# Get User's tips
url = 'https://api.foursquare.com/v2/users/user_id/tips?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION&limit=LIMIT'

In [29]:
user_id="7835045"
url = f'https://api.foursquare.com/v2/users/{user_id}/tips?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&limit={LIMIT}'
result=requests.get(url).json()
df=json_normalize(result['response']['tips']['items'])

In [30]:
df.columns

Index(['agreeCount', 'authorInteractionType', 'canonicalUrl', 'createdAt',
       'disagreeCount', 'id', 'lastUpvoteTimestamp', 'lastVoteText',
       'likes.count', 'likes.groups', 'logView', 'photo.createdAt',
       'photo.height', 'photo.id', 'photo.prefix', 'photo.source.name',
       'photo.source.url', 'photo.suffix', 'photo.tags', 'photo.visibility',
       'photo.width', 'photourl', 'text', 'todo.count', 'type',
       'venue.categories', 'venue.delivery.id',
       'venue.delivery.provider.icon.name',
       'venue.delivery.provider.icon.prefix',
       'venue.delivery.provider.icon.sizes', 'venue.delivery.provider.name',
       'venue.delivery.url', 'venue.id', 'venue.location.address',
       'venue.location.cc', 'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.formattedAddress',
       'venue.location.labeledLatLngs', 'venue.location.lat',
       'venue.location.lng', 'venue.location.neighborhood',
       'venue.location

In [31]:
pd.set_option('display.max_colwidth', -1)
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = df.loc[:, filtered_columns]

In [32]:
tips_filtered

,text,agreeCount,disagreeCount,id
0,"Came back for summer restaurant week (after a bad experience during their winter menu). The staff was lovely, food was amazing, and everything went smoothly. They definitely redeemed themselves.",0,0,5b5b645bf96b2c002c95f63b
1,Delicious soft serve and very unique ice cream flavors. They even have a special cup of whipped cream for dogs! Super friendly staff. I will be back!,0,0,5b5a19f5ccad6b002cebf801
2,Try the bucket of tots with the mac and cheese on top! I also loved the curly fries with their BBQ sauce.,0,0,5b5a18b9898bdc002cc91ef7
3,What an awesome bar. Go allll the way to the back room. The ambiance is amazing. Beautiful lights and a more quiet back bar. Great place for a date!,0,0,5b590075f870fd0039e9fddd
4,"Dining in is a much better experience than ordering out. The food quality is great when you eat there. Not sure why they use different chicken and rice for takeout, but they do. Good sesame chicken!",0,0,5b54fad81ffed7002c22c7ff
5,"Wonderful service and really cheap breakfast specials! $6.55 for eggs, bacon, potatoes, toast, coffee & juice. It might be the best deal on the UES!",0,0,5b54499290d1ed003927c461
6,"Every Saturday on the Upper East Side! Really fresh produce, baked goods, fish and more. Also a great place to get flower arrangements.",0,0,5b5447f12d270e001ccb409d
7,"They finally have a lunch menu! I know they always had happy hour specials, but the $10 lunch specials are great! Choose an appetizer and an entree. Love their tacos and margaritas.",0,0,5b52544a88a48b0024fddd82
8,Awesome addition to the neighborhood! Really good artisan ice cream. It’s a cozy little spot but definitely a good place to come for dessert.,3,0,5b52537223a2e6002c283642
9,"Limited seating, they permanently got rid of WiFi, and their coffee is overpriced. I’m sorry, what is the appeal of this place?",0,0,5b4d520ebe7078002c1d3883


# Get User's friends

In [34]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

,bio,firstName,gender,homeCity,id,lastName,lists.groups,photo.prefix,photo.suffix,superuser,tips.count
0,,Jen,female,"New York, NY",48519984,Pace,"[{'type': 'created', 'count': 2, 'items': []}]",https://igx.4sqi.net/img/user/,/48519984-I3FRIHLO1CXJVKB1.jpg,NaN,6
1,DJ. MC. Self-Proclaimed #GAMECHANGER. Polish #3BlocksFromCompton,Eva,female,"Los Angeles, CA",9876567,Magazine,"[{'type': 'created', 'count': 14, 'items': []}]",https://igx.4sqi.net/img/user/,/-DPSN5MT5KHHKELDB.png,NaN,36
2,,Meg,female,"New York, NY",10754100,Toth,"[{'type': 'created', 'count': 2, 'items': []}]",https://igx.4sqi.net/img/user/,/10754100-1DAFKBFQX2CV0OOG.jpg,NaN,4
3,"There are good ships and there are bad ships, but the best ships, are friendships.",Brian,male,NYC,305636,H,"[{'type': 'created', 'count': 11, 'items': []}]",https://igx.4sqi.net/img/user/,/1RSNFSXT4KWAPKYV.jpg,2.0,35
4,,Caitlin,female,"Brooklyn, NY",2561568,Tremblay,"[{'type': 'created', 'count': 2, 'items': []}]",https://igx.4sqi.net/img/user/,/2561568-K4OY4CZYIO1OWP2V.jpg,NaN,17
5,,Charlie,male,,94687495,Katz,"[{'type': 'created', 'count': 2, 'items': []}]",https://igx.4sqi.net/img/user/,/94687495-IV13IUV4YJXSZ5LT.jpg,NaN,0
6,,Dana,female,"Brookfield, CT",11592659,Murphy,"[{'type': 'created', 'count': 2, 'items': []}]",https://igx.4sqi.net/img/user/,/3UZXKFP2SEAT53HL.jpg,NaN,3
7,,Jonathan,male,,38608670,Levine,"[{'type': 'created', 'count': 2, 'items': []}]",https://igx.4sqi.net/img/user/,/38608670-LDC2OBIODMIEFSF5.jpg,NaN,0
8,,Joe,male,"Los Angeles, CA",6969890,Rome,"[{'type': 'created', 'count': 2, 'items': []}]",https://igx.4sqi.net/img/user/,/POCA4JWZR5TU3WX2.jpg,NaN,38
9,,Lisa,female,"New York, New York",2339403,Kirshner,"[{'type': 'created', 'count': 4, 'items': []}]",https://igx.4sqi.net/img/user/,/2OVSQ2GBNEBTXHV4.jpg,NaN,4


In [36]:
user_friends['firstName']

0    Jen     
1    Eva     
2    Meg     
3    Brian   
4    Caitlin 
5    Charlie 
6    Dana    
7    Jonathan
8    Joe     
9    Lisa    
Name: firstName, dtype: object

# Retrieve the User's Profile Image

In [37]:
user_data['photo']

{'prefix': 'https://igx.4sqi.net/img/user/',
 'suffix': '/7835045_vzeT9M7E_W9mOqaLB2Kjg0RdSk9ZaZ3qumWd83vX699ukd2xQKig6s29t8WuzPpeBW54z34H4.jpg'}

In [47]:
Image(url='https://igx.4sqi.net/img/user/300x300/7835045_vzeT9M7E_W9mOqaLB2Kjg0RdSk9ZaZ3qumWd83vX699ukd2xQKig6s29t8WuzPpeBW54z34H4.jpg')

# Explore a location
https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT

In [19]:
latitude = 37.7866329 
longitude = -122.41195210013
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=L0XUK3OF0DEMWCIMYXLNQEDWMM1GIBC43PQCFYYID2XGX4WV&client_secret=FYVOYA1ZZ3YQ320RWOU5XCVD1Y3TA1BCTGIW2OS5ZJTG4CKH&ll=37.7866329,-122.41195210013&v=20180731&radius=500&limit=30'

In [30]:
results = requests.get(url).json()
len(results['response']['groups'][0]['items'])
'There are {} around The Marker Hotel.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around The Marker Hotel.'

In [31]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4a6f3531f964a5209cd51fe3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d175941735',
    'name': 'Gym / Fitness Center',
    'pluralName': 'Gyms or Fitness Centers',
    'primary': True,
    'shortName': 'Gym / Fitness'}],
  'id': '4a6f3531f964a5209cd51fe3',
  'location': {'address': '524 Post St',
   'cc': 'US',
   'city': 'San Francisco',
   'country': 'United States',
   'crossStreet': 'at Taylor St',
   'distance': 189,
   'formattedAddress': ['524 Post St (at Taylor St)',
    'San Francisco, CA 94102',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.788181056861674,
     'lng': -122.41106656038573}],
   'lat': 37.788181056861674,
   'lng': -122.41106656038573,
   'postalCode': '9410

In [32]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Olympic Club,Gym / Fitness Center,524 Post St,US,San Francisco,United States,at Taylor St,189,"[524 Post St (at Taylor St), San Francisco, CA...","[{'label': 'display', 'lat': 37.78818105686167...",37.788181,-122.411067,NaN,94102,CA,4a6f3531f964a5209cd51fe3
1,Tselogs,Filipino Restaurant,552 Jones St,US,San Francisco,United States,at Geary St,104,"[552 Jones St (at Geary St), San Francisco, CA...","[{'label': 'display', 'lat': 37.78647937415373...",37.786479,-122.413128,NaN,94102,CA,56a7f7fa498e29aaf707c5d4
2,Ryoko's Japanese Restaurant & Bar,Sushi Restaurant,619 Taylor St,US,San Francisco,United States,at Cosmo Pl,172,"[619 Taylor St (at Cosmo Pl), San Francisco, C...","[{'label': 'display', 'lat': 37.788183, 'lng':...",37.788183,-122.411882,NaN,94102,CA,433c8000f964a52043281fe3
3,Bourbon & Branch,Speakeasy,501 Jones St,US,San Francisco,United States,at O'Farrell St,135,"[501 Jones St (at O'Farrell St), San Francisco...","[{'label': 'display', 'lat': 37.78582823420628...",37.785828,-122.413115,NaN,94102,CA,4617e746f964a52051451fe3
4,Wilson & Wilson Private Detective Agency,Speakeasy,501 Jones St,US,San Francisco,United States,at O'Farrell,116,"[501 Jones St (at O'Farrell), San Francisco, C...","[{'label': 'display', 'lat': 37.78596942443605...",37.785969,-122.412969,NaN,94102,CA,4d7b1c3e7a1c6dcb57d2ca61
5,Benjamin Cooper,Cocktail Bar,398 Geary St,US,San Francisco,United States,Mason St,203,"[398 Geary St (Mason St), San Francisco, CA 94...","[{'label': 'display', 'lat': 37.787396, 'lng':...",37.787396,-122.409847,NaN,94102,CA,54f132e7498e5d065370a6b1
6,American Conservatory Theater,Theater,405 Geary St,US,San Francisco,United States,at Mason,166,"[405 Geary St (at Mason), San Francisco, CA 94...","[{'label': 'display', 'lat': 37.78696847403834...",37.786968,-122.410103,NaN,94102,CA,4ab6d922f964a520497920e3
7,Pearl's Deluxe Burgers,Burger Joint,708 Post St,US,San Francisco,United States,btwn Jones & Leavenworth St,185,"[708 Post St (btwn Jones & Leavenworth St), Sa...","[{'label': 'display', 'lat': 37.78764896810783...",37.787649,-122.413620,NaN,94109,CA,49c6844cf964a52054571fe3
8,Le Colonial,Vietnamese Restaurant,20 Cosmo Pl,US,San Francisco,United States,from Taylor St.(btwn Post & Sutter),188,[20 Cosmo Pl (from Taylor St.(btwn Post & Sutt...,"[{'label': 'display', 'lat': 37.78828061940812...",37.788281,-122.412450,NaN,94109,CA,40dcbc80f964a52085011fe3
9,Taylor Street Coffee Shop,Coffee Shop,375 Taylor St,US,San Francisco,United States,at O'Farrell,124,"[375 Taylor St (at O'Farrell), San Francisco, ...","[{'label': 'display', 'lat': 37.78568085058454...",37.785681,-122.411216,NaN,94102,CA,4a671dbaf964a520edc81fe3


In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='The Marker',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

# Explore Trending Venues

Trending venues are the places around the given latitude and longitude those have the highest foot traffic at this moment it gets updated frequently.

https://api.foursquare.com/v2/venues/trending?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION

In [35]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b6388d7dd5797468445e7dd'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/conventioncenter_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1ff931735',
      'name': 'Convention Center',
      'pluralName': 'Convention Centers',
      'primary': True,
      'shortName': 'Convention Center'}],
    'id': '4cc2f44942d1b60cbdc81313',
    'location': {'address': '747 Howard St',
     'cc': 'US',
     'city': 'San Francisco',
     'country': 'United States',
     'crossStreet': 'at Moscone Center',
     'distance': 979,
     'formattedAddress': ['747 Howard St (at Moscone Center)',
      'San Francisco, CA 94103',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.78417067388567,
       'lng': -122.40126161270385}],
     'lat': 37.78417067388567,
     'lng': -122.40126161270385,
     'postalCode': '94103',
     'state': 'CA'},
    'name': 'Moscone So

In [36]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)
trending_venues_df    

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Moscone South,Convention Center,979,San Francisco,94103,CA,United States,37.784171,-122.401262
1,Creator,Burger Joint,1207,San Francisco,94107,CA,United States,37.784284,-122.398556


In [37]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
venues_map